# Chain

**Chain**(체인)은 여러 컴포넌트(요소)를 정해진 순서대로 연결하여 **복잡한 AI 작업을 단계별로 자동화**할 수 있도록 돕는 구조이다.

- 각 컴포넌트는 입력을 받아 특정 처리를 수행한 후 다음 단계로 결과를 전달한다.
- 복잡한 작업을 여러 개의 단순한 단계로 나누고, 각 단계를 순차적으로 실행함으로써 전체 작업을 체계적으로 구성할 수 있다.

## 기본 개념

- 체인은 하나의 LLM 호출에 그치지 않고 **여러 LLM 호출이나 도구 실행을 순차적으로 연결**할 수 있다.
- 예를 들어, 사용자의 질문 → 검색 → 요약 → 응답 생성 같은 일련의 작업을 체인으로 구성할 수 있다.
- 체인을 사용하면 코드의 재사용성과 유지 보수성이 향상된다.

## LangChain에서의 Chain 구성 방식

LangChain은 다음 두 가지 방식을 통해 체인을 구성할 수 있다.

### 1. Off-the-shelf Chains 방식 (클래식 방식)

- LangChain에서 제공하는 **미리 정의된 Chain 클래스**(예: `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)를 활용하는 방식이다.
- 이 방식은 LangChain의 **초기 구조**이며, 대부분의 클래스는 현재 **더 이상 사용되지 않음(deprecated)** 상태이다.

> 현재 LangChain에서는 이 방식을 권장하지 않는다.

### 2. LCEL (LangChain Expression Language) 방식

- 체인을 함수형 방식으로 선언할 수 있는 **표현식 기반의 체인 구성 언어**이다.
- LCEL 방식은 간결하고 선언적인 문법을 제공하여 **직관적이고 확장성 있는 체인 구성**이 가능하다.
- `Runnable`이라는 공통 인터페이스를 기반으로 다양한 요소를 조합하여 체인을 구성한다.
- 체인의 각 구성 요소는 `invoke()` 메서드로 실행된다.

In [43]:
# off-the-shelf 방식
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
load_dotenv()

prompt_template = PromptTemplate(template="{item}에 어울리는 이름 {count}개를 만들어 주세요")

model = ChatOpenAI(model_name= "gpt-4o-mini")
parser = StrOutputParser()

# 변수:값 -> (prompt_template) -prompt -> (model) -> 응답결과 -> (parser) -> 최종 결과


In [44]:
from langchain import LLMChain
chain = LLMChain(
    prompt=prompt_template,
    llm  =model,
    output_parser =parser
)
response = chain.invoke({"item":"가방", "count":5})

In [45]:
print(response["text"])

물론이죠! 여러분의 가방에 어울릴만한 이름 5개를 제안해 드릴게요:

1. **모던 미니멀** - 세련되고 심플한 디자인에 잘 어울리는 이름.
2. **여행의 친구** - 여행 가방이나 캐리어에 적합한 이름.
3. **스타일 아이콘** - 패셔너블한 디자인의 가방에 어울리는 이름.
4. **데일리 베이직** - 일상적인 사용에 적합한 기본적인 가방을 위한 이름.
5. **자연의 영감** - 친환경 소재나 자연에서 영감을 받은 디자인의 가방에 잘 어울리는 이름.

어떤 이름이 마음에 드시나요? 추가로 더 필요하시면 말씀해 주세요!


# [LCEL](https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel) (LangChain Expression Language)
- LCEL은 LangChain의 핵심 기능인 체인(Chain)을 더욱 간결하고 유연하게 구성할 수 있도록 고안된 **선언형 체인(chain) 구성 언어**이다.
- 파이프 연산자 `|`를 사용해 선언적 방법으로 여러 작업을 연결한다.
- 체인을 구성하는 각 요소는 `Runnable` 타입으로, 체인 내에서 실행 가능한 단위이다.
- 각 단계는 invoke() 메서드를 통해 실행되며, 앞 단계의 출력이 다음 단계의 입력으로 자동 전달된다.
    - [Runnable 컴포넌트별 입출력 타입](https://python.langchain.com/docs/concepts/runnables/#input-and-output-types)
    - 각 컴포넌트의 input과 output 타입에 맞춰 값이 전달되도록 한다.
- https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

## [Runnable](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html)
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 **체인(chain) 에 포함 되어**  복잡한 작업의 각 단계를 수행 한다.
- Chain을 구성하는 class들은 Runnable의 상속 받아 구현한다.
- **Prompt Template클래스**, **Chat 모델, LLM 모델 클래스**, **Output Parser 클래스** 등 다양한 컴포넌트가 Runnable을 상속받아 구현된다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 호출, 출력 파싱 등)을 수행하는 독립적인 컴포넌트이다.
    - 각 컴포넌트는 독립적으로 테스트 및 재사용이 가능하며, 조합하여 복잡한 체인을 구성할 수 있다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 체인(chain, 일련의 연결된 작업 흐름)을 구성하는 기본 단위로 사용된다.
    - LangChain Expression Language(LCEL)를 사용하면 | 연산자를 통해 여러 Runnable을 쉽게 연결할 수 있다.
    - 력과 출력의 형식을 일관되게 유지하여 각 단계가 자연스럽게 연결된다.
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 문제 발생 시 어느 단계에서 오류가 발생했는지 쉽게 확인할 수 있다.
    - 복잡한 작업을 작은 단위로 나누어 체계적으로 관리할 수 있다.
      
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
    - `invoke()`: 단일 입력을 처리하여 결과를 반환.
    - `batch()`: 여러 입력 데이터들을 한 번에 처리.
    - `stream()`: 입력에 대해 스트리밍 방식으로 응답을 반환.
    - `ainvoke()`: 비동기 방식으로 입력을 처리하여 결과를 반환.

### Runnable의 주요 구현체(하위 클래스)

- `RunnableSequence`
    - 여러 `Runnable`을 순차적으로 연결하여 실행하는 구성이다.
    - 각 단계의 출력이 다음 단계의 입력으로 전달된다.
    - LCEL을 사용하여 체인을 구성할 경우 자동으로 `RunnableSequence`로 변환된다.
-  `RunnablePassThrough`
    - 입력 데이터를 가공하지 않고 그대로 다음 단계로 전달하는 `Runnable`이다.
    - 선택적으로 미리 정의된 키-값 쌍을 함께 전달할 수 있다.

- `RunnableParallel`
    - 여러 `Runnable`을 병렬로 실행한 후, 결과를 결합하여 다음 단계로 전달한다.
    - 병렬 처리를 통해 처리 속도를 개선할 수 있다.

- `RunnableLambda`
    - 일반 함수 또는 `lambda` 함수를 `Runnable`로 변환하여 체인에 포함할 수 있다.
    - 사용자 정의 함수로 동작을 확장할 때 유용하다.

#### Runnable 예제

In [13]:
from dotenv import load_dotenv
load_dotenv()

from langchain_core.runnables import Runnable
#사용자 정의
class MyRunnalble(Runnable):
    def invoke(self, input_data:str, config:dict=None):
        #invoke(): 구현하는 Runnable이 해야하는 작업을 구현하는 메서드
        #input_data : 입력값
        #config : 일할 떄 필요한 설정값들.
        if config is not None and config.get('lang') =='en':
            return f"Explain {input_data} in one sentense"
        return f"{input_data}에 대해서 한 문장으로 설명해줘"


In [15]:
my_runnable = MyRunnalble()
my_runnable.invoke("사과")
my_runnable.invoke("사과")
my_runnable.invoke("Apple", {"lang":'en'})

'Explain Apple in one sentense'

In [18]:
from langchain_openai import ChatOpenAI

my_runnalble = MyRunnalble()
model = ChatOpenAI(model_name="gpt-4o-mini")

prompt = my_runnable.invoke("Apple",{"lang":"en"})
prompt = my_runnable.invoke("Lanchain")
response = model.invoke(prompt)
print(response)

content='Langchain은 다양한 언어 모델을 활용하여 자연어 처리 및 대화형 AI 애플리케이션을 구축하는 데 필요한 도구와 프레임워크를 제공하는 라이브러리입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 19, 'total_tokens': 63, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgjKfvZiAFeW42tqum2T6BUNP6CiW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--98513a70-caf8-4a24-be1d-dac892249988-0' usage_metadata={'input_tokens': 19, 'output_tokens': 44, 'total_tokens': 63, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [20]:
# chain -> Runnable | Runnable | Runnable
chain = my_runnable | model
# chain 호출 --> invoke
res = chain.invoke("과일 배")
print(res.content)

배는 아삭하고 수분이 풍부한 과일로, 주로 가을에 수확되며 달콤한 맛과 다양한 품종이 특징입니다.


In [22]:
# 기본 체인 구성 : prompt_template -> model -> output parser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# role : system, user/human, ai/assistant
prompt_template = ChatPromptTemplate(
    messages=[
        ("system","당신은 오랜 경력의 한국관광가이드입니다. 여행객들에게 설명하듯이 답변을 해주세요."),
        ("human","{query}")
    ]
)
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=1.0)

guide_chain = prompt_template | model | StrOutputParser()
print(type(guide_chain)) # RunnableSequence:Runnable 타입
                         # => chain도 다른 chain을 구성요소로 포함할 수 있다.




<class 'langchain_core.runnables.base.RunnableSequence'>


In [25]:
query = "서울에서 꼭 가봐야되는 여행지를 세 곳만 알려줘."
response = guide_chain.invoke({"query":query})
print(response)

서울은 매력적인 여행지로 가득 차 있습니다. 그중에서도 꼭 가봐야 할 세 곳을 소개해드리겠습니다.

1. **경복궁**: 서울의 대표적인 고궁인 경복궁은 조선왕조의 첫 번째 궁궐로, 아름다운 전통 건축과 함께 역사의 흔적을 간직하고 있습니다. 또한, 근처에 국립민속박물관과 국립고궁박물관도 있어 한국의 역사와 문화를 더 깊게 이해할 수 있습니다. 궁궐 내의 수문장 교대식은 꼭 관람해보시기 바랍니다!

2. **남산서울타워 (N서울타워)**: 서울의 랜드마크 중 하나인 남산서울타워는 서울 시내를 한눈에 내려다볼 수 있는 최고의 전망대를 제공합니다. 특히 해질 무렵에 방문하시면 아름다운 노을을 감상하며 로맨틱한 시간을 보낼 수 있습니다. 타워 주변의 남산 공원에서 산책도 즐기실 수 있어요.

3. **명동**: 한국의 대표적인 쇼핑과 먹거리의 중심지인 명동은 패션, 화장품, 그리고 다양한 길거리 음식을 즐길 수 있는 곳입니다. 화려한 네온사인과 함께 다양한 먹거리를 경험하시며 쇼핑도 만끽할 수 있습니다. 저녁이면 거리 공연도 잊지 마세요!

이 세 곳은 서울의 다양한 매력을 느낄 수 있는 장소들이니 꼭 방문해 보시길 추천드립니다!


In [28]:
query = input("질문")
while True:
    if query == '!quit':
        break
    resp = guide_chain.invoke({"query":query})
    print("User:", query)
    print("AI:", resp)
    print("-"*50)

User: 
AI: 안녕하세요! 한국 관광에 대해 궁금한 점이 있으시면 무엇이든 물어보세요. 서울의 역사적인 장소에서부터 아름다운 자연경관, 맛있는 음식까지 다양한 정보를 제공해 드리겠습니다. 어떤 여행 계획을 세우고 계신가요?
--------------------------------------------------


KeyboardInterrupt: 

#### RunnableLambda 예제

In [29]:
from langchain_core.runnables import RunnableLambda
# RunnableLammda(함수) -> 함수를 실행하는 Runnable을 생성. 
my_runnable2 = RunnableLambda(lambda input_data:f"{input_data}를 한 문장으로 설명해줘.")
my_runnable2.invoke("LLM")

'LLM를 한 문장으로 설명해줘.'

In [30]:
chain = my_runnable2 | model
chain.invoke("LLM")

AIMessage(content='LLM(대규모 언어 모델)은 대량의 텍스트 데이터를 기반으로 학습하여 자연어 이해 및 생성 작업을 수행하는 인공지능 모델입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 18, 'total_tokens': 55, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgkAGmhWDMnTkzhslnaicegpXowFz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--574d071a-ba9d-4d4d-adc4-cc7d02c984ba-0', usage_metadata={'input_tokens': 18, 'output_tokens': 37, 'total_tokens': 55, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### RunnablePassThrough 예제

In [32]:
# 1 앞 Runnable이 처리한 결과를 다음 Runnable에 그대로 전달
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke("안녕")
RunnablePassthrough().invoke({"key":"value"})

{'key': 'value'}

In [37]:
# 1 앞 Runnable이 처리한 결과에 Item을 추가해서 다음 Runnable에 전달.
# -> 입력으로 Dictionary를 받아서 거기에 item을 추가.
# RunnablePassthrough.assign(key=Runnable)
# -받은 dictionary에 "key1":Runnable반환값, "key2":Runnable반환값, .. 추가해서 다음으로 전달
address_runnable = RunnableLambda(lambda x:"서울시 금천구") #"서울시 금천구"를 반환
phone_runnable = RunnableLambda(lambda x:"010-2001-2301") 

RunnablePassthrough().assign(address=address_runnable, phone=phone_runnable).invoke({"name":"홍길동"})

{'name': '홍길동', 'address': '서울시 금천구', 'phone': '010-2001-2301'}

#### RunnableSequence 예제

In [38]:
from langchain_core.runnables import RunnableSequence

run1 = RunnableLambda(lambda x:x+1)
run2 = RunnableLambda(lambda x:x*2)

chain = run1 | run2
chain.invoke(30)

62

In [39]:
chain2 = RunnableSequence(run1, run2)
chain2.invoke(100)


202

#### RunnableParallel 예제

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

run1 = RunnableLambda(lambda x:x+1)
run2 = RunnableLambda(lambda x:x*2)
run3 = RunnableLambda(lambda x:x//3)

chain = RunnableParallel(
    {
        "result1":run1,
        "result2":run2,
        "result3":run3
    }
)

# 각 Runnable들을 각각 실행하고 그 결과를 key에 할당한 Dictionary를 반환.


#### LCEL Chain 예제

In [81]:
# 음식 이름을 받아서 레시피를 "영어"로 출력하는 chain을 구성
# prompt template -> model -> output parser

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from textwrap import dedent


input_data = input()
prompt_template = PromptTemplate(
    template=dedent("""
    #Instruction
    당신은 숙련된 요리 연구가입니다. 요청한 음식의 레시피를 작성해 주세요
    
    #Input data
    음식이름 : {food}
         
    """)
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=1.0)

food_chain = prompt_template | model | StrOutputParser()

response = food_chain.invoke({"food":input_data})
print(response)

## 돼지국밥 레시피

### 재료
- 돼지고기(목살 또는 사태) 500g
- 대파 2대
- 마늘 5쪽
- 생강 1조각
- 멸치 육수 또는 물 2L
- 고추가루 2큰술
- 소금, 후추 적당량
- 흰밥 4공기
- 부추 또는 다진 파 약간 (고명용)
- 깨소금 약간 (선택)

### 조리 방법

1. **돼지고기 준비하기**  
   - 돼지고기는 깨끗이 씻어서 2~3cm 크기로 잘라줍니다.  
   - 냄비에 돼지고기를 넣고 물을 부어 중불에서 끓여 준비합니다. 고기가 삶아질 때 겉면의 불순물이 나올 수 있으니 거품을 제거합니다.

2. **육수 만들기**  
   - 다른 냄비에 멸치와 다시마를 넣고 2리터의 물을 넣어 끓입니다.  
   - 끓기 시작하면 중불로 줄여 20~30분간 우려낸 후, 건더기는 버리고 육수만 남겨둡니다.  
   - 멸치 육수가 없다면 물만 사용해도 괜찮습니다.

3. **고기와 육수 끓이기**  
   - 끓여낸 돼지고기를 건져내고, 고기가 익고 부드러워질 때까지 다시 냄비에 육수와 함께 넣어서 중불로 끓여줍니다.  
   - 대파와 마늘, 생강을 넣고, 고추가루도 추가해줍니다. 소금과 후추로 간을 맞춰주시고, 약 1시간 정도 끓입니다. 고기가 잘 익으면 국물이 진해집니다.

4. **재료 준비하기**  
   - 고기가 익는 동안 흰밥을 준비합니다.  
   - 부추나 다진 파를 씻어서 준비합니다.

5. **서빙하기**  
   - 국물이 충분히 진하고 고기가 부드러워지면 불을 끄고, 국밥을 그릇에 담습니다.  
   - 밥 위에 고기를 올리고, 국물을 적당히 부어줍니다.  
   - 부추 또는 다진 파로 고명을 올리고, 원하시면 깨소금을 뿌려줍니다.

6. **맛있게 먹기**  
   - 뜨거운 국밥을 한 그릇 더하고, 김치나 깍두기와 함께 즐겨보세요!

이렇게 완성된 돼지국밥은 깊은 맛과 풍미가 어우러져 가족 모두가 좋아할 만한 요리입니다. 맛있게 드세요!


In [71]:
# 번역 chain -> 입력된 내용을 지정한 언어로 번역하는 체인
# prompt template -> model -> output parser

In [80]:
prompt = PromptTemplate(
    input_variables=["content", "language"],
    template=dedent("""    
    # Instruction
    당신은 다국어가 가능한 숙련된 번역가다.
    요청된 문장을 {language}로 번역해줘.

    # Input data(번역할 내용)
    {content}
                    
    # Output Indicator
    - 번역한 내용만 출력해 줘
    """
    )
)

translate_chain = prompt | model | StrOutputParser()

res = translate_chain.invoke({"content":response, "language":"일본어"})
# 안녕하세요 -> 독일어로 번역한 응답
print(res)

### アーリオ・オーリオパスタレシピ

#### 材料:
- スパゲッティ400g（またはお好みのパスタ）
- ニンニク6片、薄切り
- エクストラバージンオリーブオイル100ml
- 赤唐辛子フレーク小さじ1（お好みで調整）
- 塩、適量
- 新鮮な黒胡椒、適量
- 刻んだ新鮮なパセリ（飾り用）
- おろしたパルメザンチーズ（お好みで）

#### 作り方:

1. **パスタを茹でる:**
   - 大きな鍋に塩水を沸かします。パッケージの指示に従ってスパゲッティをアルデンテになるまで茹でます。約1カップのパスタの茹で汁を取っておき、パスタを水切りします。

2. **ニンニクオイルを作る:**
   - 大きなフライパンにエクストラバージンオリーブオイルを中火で熱します。薄切りのニンニクを加え、約1〜2分間、ニンニクが薄く黄金色になり香りが立つまで炒めます。ニンニクが焦げないように注意してください。焦げると苦味が出ます。

3. **赤唐辛子フレークを加える:**
   - 赤唐辛子フレークを加え、追加で30秒間調理して風味を融合させます。

4. **パスタとソースを組み合わせる:**
   - 火を弱め、水切りしたパスタをニンニクオイルのフライパンに加えます。オイルとニンニクでパスタが均等にコーティングされるように和えます。パスタが乾燥している場合は、徐々に取っておいたパスタの水を加えて、お好みの濃度になるまで調整します。

5. **味付け:**
   - 塩と新鮮な黒胡椒で味を調えます。再度和えて混ぜます。

6. **盛り付け:**
   - 火から下ろし、刻んだ新鮮なパセリとおろしたパルメザンチーズ（お好みで）で飾ります。すぐに提供し、楽しんでください！

#### ヒント:
- より風味を増すために、提供前にレモンの皮や新鮮なレモンジュースを少し加えることができます。
- さらにボリュームを加えるために、エビや鶏肉などの炒めた野菜やタンパク質を追加することもできます。 

自家製アーリオ・オーリオパスタを楽しんでください！


## Chain과 Chain간의 연결

In [84]:
from langchain_core.runnables import RunnableParallel
# food_chain + translate_chain
# food_chain_prompt: 변수 - food
# translate_chain : 변수 - language, content
RunnableParallel({"key":Runnable})
# LCEL에서 RunnableParallel => {"key":Runnable,"key2":Runnable}
chain = {"content":food_chain, 
         "language":RunnableLambda(lambda x : x['language'])} | translate_chain 

In [ ]:
res = chain.invoke({"food":"냉면", "language": "러시아어"})
print(res)

## Рецепт холодной лапши

### Ингредиенты
#### Лапша
- Холодная лапша 200 г (гречневая лапша или пшеничная лапша)
  
#### Бульон
- Холодная вода 4 чашки
- Анчоусы 1/2 чашки
- Ламинария 1 лист (10 см)
- Соевый соус 4 столовые ложки
- Уксус 3 столовые ложки
- Сахар 2 столовые ложки
- Соль по вкусу
- Чеснок (мелко нарезанный) 1 столовая ложка
- Перец чили порошок (по желанию) 1 столовая ложка

#### Гарнир
- Огурец 1/2 штуки (нарезанный соломкой)
- Вареное яйцо 1 штука (разрезать пополам)
- Груша 1/4 штуки (нарезанная соломкой)
- Нори (морская водоросль) немного
- Мелко нарезанный зеленый лук немного (по желанию)
- Кунжутная соль немного (по желанию)

### Способ приготовления

1. **Приготовление бульона**:
   - В кастрюле налейте холодную воду и добавьте анчоусы и ламинарии.
   - Когда начнет кипеть на среднем огне, уменьшите огонь и варите около 20 минут.
   - Выключите огонь и удалите анчоусы и ламинарию, затем добавьте соевый соус, уксус, сахар, нарезанный чеснок и соль по вкусу.
   - О

: 

# 사용자 함수를 Chain에 적용하기

## 사용자 함수를 Runnable로 정의 (RunnableLambda)
- 임의의 함수를 Runnable로 정의 할 수있다.
  - chain에 포함할 기능을 함수로 정의할 때 주로 사용. 
- `RunnableLambda(함수)` 사용
  - 함수는 invoke() 메소드를 통해 입력받은 값을 받을 **한개의 파라미터**를 선언해야 한다.

## 사용자 함수를 Chain으로 정의
- Chain 을 구성하는 작업 사이에 추가 작업이 필요할 경우, 중간 결과를 모두 사용해야 하는 경우 함수로 구현한다.
- `@chain` 데코레이터를 사용해 함수에 선언한다.

### Runnable 에 사용할 **사용자 정의 함수** 구문
- 이전 Chain의 출력을 입력 받는 **파라미터를 한개** 선언한다. (첫번째 파라미터)
- `invoke()`로 호출 할때 전달 하는 추가 설정을 입력받는 파라미터를 선언한다.(두번째 파라미터 - Optional)
  - RunnableConfig 타입의 값을 받는데 Dictionary 형식으로 `{"configuable": {"설정이름":"설정값"}}` 형식으로 받는다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
  ```python
  def plus(num1, num2):
      ...

  def wrapper_plus(nums:dict|list):
      return plus(nums['num1'], nums['num2'])
  ```

# Cache

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
  
    ```python
    set_llm_cache(Cache객체)
    ```